# TechQuake: Behavior of Major Tech Stocks During Market Shocks

This notebook explores the behavior of selected technology-related stocks:

- AAPL (Apple)
- MSFT (Microsoft)
- NVDA (Nvidia)
- AMZN (Amazon)
- SPOT (Spotify)

Using daily price data from Yahoo Finance (`yfinance`), the notebook:

1. Fetches historical price data for the selected tickers.
2. Cleans and transforms the data using helper functions in `stock_data/`.
3. Computes daily returns, cumulative returns (indexed to 100), rolling volatility, and drawdowns.
4. Visualizes performance and risk, with an emphasis on market shocks such as the COVID-19 period and the 2022 inflation-driven selloff.

The goal is to compare how different tech companies behave under stress, both in terms of long-term growth and downside risk.

In [1]:
# Add the parent directory to the system path for proper module imports
import sys
sys.path.append("..")

In [2]:
# Project imports: data loading, processing, and plotting
from stock_data.fetch_data import fetch_stock_data
from stock_data.process_data import (
    extract_price_table,
    compute_daily_returns,
    compute_cumulative_returns,
    compute_rolling_volatility,
    compute_drawdowns,
)
from stock_data.visualizations import (
    plot_cumulative_returns,
    plot_stock_comparison,
    plot_volatility_with_events,
)

from datetime import datetime

## Inspecting the raw price data

I start by downloading daily OHLCV data for five large tech and media stocks.  
Below I check a sample of the dataset and its structure to make sure the download worked as expected.

In [12]:
# Define the tickers to analyse
tickers = ["AAPL", "MSFT", "NVDA", "AMZN", "SPOT"]

# Choose the start date for the analysis
start_date = "2018-01-01"

# Fetch raw historical data from Yahoo Finance
raw_data = fetch_stock_data(tickers, start_date=start_date)

# Preview the first few rows of the raw dataset
raw_data.head()

Price        Date      Close                                            High  \
Ticker                  AAPL       AMZN       MSFT      NVDA SPOT       AAPL   
0      2018-01-02  40.341888  59.450500  79.050369  4.928807  NaN  40.351258   
1      2018-01-03  40.334869  60.209999  79.418274  5.253192  NaN  40.878196   
2      2018-01-04  40.522228  60.479500  80.117279  5.280884  NaN  40.625273   
3      2018-01-05  40.983578  61.457001  81.110558  5.325634  NaN  41.070228   
4      2018-01-08  40.831348  62.343498  81.193321  5.488815  NaN  41.126429   

Price                                   ...       Open                        \
Ticker       AMZN       MSFT      NVDA  ...       AAPL       AMZN       MSFT   
0       59.500000  79.381471  4.932516  ...  39.850088  58.599998  79.215920   
1       60.274502  79.565433  5.283603  ...  40.405128  59.415001  79.151553   
2       60.793499  80.623131  5.391155  ...  40.407473  60.250000  79.639018   
3       61.457001  81.312898  5.362968  ...  40.618239  60.875500  80.623104   
4       62.653999  81.469242  5.562988  ...  40.831348  61.799999  81.119742   

Price                     Volume                                      
Ticker      NVDA SPOT       AAPL      AMZN      MSFT       NVDA SPOT  
0       4.840541  NaN  102223600  53890000  22483800  355616000  NaN  
1       5.046249  NaN  118071600  62176000  26061400  914704000  NaN  
2       5.334536  NaN   89738400  60442000  21912000  583268000  NaN  
3       5.295718  NaN   94640000  70894000  23407100  580124000  NaN  
4       5.449256  NaN   82271200  85590000  22113000  881216000  NaN  

[5 rows x 26 columns]

### Understanding the Raw Dataset (OHLCV)

The table above shows the raw market data fetched from Yahoo Finance.  
Each stock includes the standard OHLCV fields:

- **Open** – price at market open  
- **High** – highest price of the day  
- **Low** – lowest price of the day  
- **Close** – final price of the day (used in this project)  
- **Volume** – number of shares traded  

Only the **Close** prices will be used in the subsequent analysis (returns, volatility, drawdowns),  
but previewing the full OHLCV structure verifies that the dataset was downloaded correctly and contains complete market data.

### Understanding Missing Data for Spotify (SPOT)

In the raw dataset, the column for Spotify (SPOT) shows `NaN` values before April 2018.  
This is expected and correct.

Spotify went public on **April 3, 2018**, which means there is **no market price data available before its IPO date**.  
The other companies (AAPL, MSFT, NVDA, AMZN) were already publicly traded during this period, so they do not show missing values.

### Dataset Structure and Missing Values

The summary below shows the structure of the dataset, including the number of
rows, columns, and any missing values. This helps confirm that the data was
loaded correctly and highlights gaps (such as missing SPOT data before April
2018).

In [10]:
# Basic information about the dataset
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1989 entries, 0 to 1988
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   (Date, )        1989 non-null   datetime64[ns]
 1   (Close, AAPL)   1989 non-null   float64       
 2   (Close, AMZN)   1989 non-null   float64       
 3   (Close, MSFT)   1989 non-null   float64       
 4   (Close, NVDA)   1989 non-null   float64       
 5   (Close, SPOT)   1927 non-null   float64       
 6   (High, AAPL)    1989 non-null   float64       
 7   (High, AMZN)    1989 non-null   float64       
 8   (High, MSFT)    1989 non-null   float64       
 9   (High, NVDA)    1989 non-null   float64       
 10  (High, SPOT)    1927 non-null   float64       
 11  (Low, AAPL)     1989 non-null   float64       
 12  (Low, AMZN)     1989 non-null   float64       
 13  (Low, MSFT)     1989 non-null   float64       
 14  (Low, NVDA)     1989 non-null   float64       
 15  (Low

In [ ]:
# Extract a clean table of closing prices
prices = extract_price_table(raw_data, price_field="Close")
prices.head()

In [ ]:
# Compute daily returns for each ticker
daily_returns = compute_daily_returns(prices)
daily_returns.head()

In [ ]:
# Compute cumulative returns, starting from an index level of 100
cumulative_returns = compute_cumulative_returns(daily_returns, base=100.0)
cumulative_returns.head()

In [ ]:
# Describe how rolling volatility is calculated and what it represents. It shows how the stock price fluctuates over time.
rolling_volatility = compute_rolling_volatility(daily_returns, window=21)
rolling_volatility.tail()